**Tutorial 5** : Fuzzy Logic and Rules-Based Systems

This tutorial breaks from the others in that it does not use the provided data. Fuzzy logic (FL) in terms of optimization and modeling is generally part of a Fuzzy Logic Controller (FLC) or another system that uses FL to come to a decision. Similarly, Rules-Based systems (the less-flexible counterpart of a Fuzzy Logic system), are part of a large controller used to make decisions.

These two methods are important in terms of the **design** of the antennas we have been looking at for the last 4 tutorials. The data collected for those tutorials was from an iterative batch collection that took multiple weeks and multiple simulation machines to collect. Data can have important insights into the behavior and design process. However, with only 3 sampled designs of a single topology, this is not sufficient to adapt to unknown designs.

One potential method of addressing this problem is Reinforcement Learning (RL).

<br>

**Reinforcement learning** is a type of machine learning where an agent learns to make decisions by interacting with an environment. The goal of reinforcement learning is for the agent (software, object) to learn a policy (a set of actions) that maximizes a cumulative reward over time. The agent explores the environment, takes actions, and receives feedback (in the form of rewards or penalties) that helps it improve its behavior.  

Reinforcement learning is a type of online machine learning (Online ML). Online ML is a type of machine learning where the model is trained incrementally as new data arrives, rather than being trained all at once on a large dataset (as in traditional "batch" learning, which was seen in the previous tutorial). In online learning, the model is updated continuously as more data is collected.

However, in order to facilitate this model update, there must be some guidelines for how the model is updated. These guidelines can take several forms, but commonly a mathematical model is used to describe the behavior of the agent in the system. This includes limits to what the outputs are, and what to do if invalid inputs are entered.

A simple Rules-Based controlled system might consist of a series of IF-ELSE statements to decide on the next steps based on a small number of controllable parameters. This is useful if the number of potential actions (or changeable variables) is small.

If the system behavior is complex, then using fuzzy logic to incorporate less strict decision making policies may be beneficial. A Fuzzy Logic Controller (FLC) is a control system based on fuzzy logic rather than traditional binary (true/false) logic. It is designed to handle reasoning that is approximate rather than precise, and it works well in situations where uncertainty, imprecision, or vagueness exists in the system being controlled. In contrast to classical control systems (which rely on precise, numerical input), fuzzy logic allows for a more human-like way of reasoning, making it ideal for complex systems where exact mathematical models are difficult or impossible to derive.


Demonstrated below are the following:

* An example of a rules-based system with 3 inputs, 1 decision
* An example of a fuzzy logic controller with 3 inputs, 1 decision


Both examples require several assumptions or information about:
1. The environment
2. Potential actions
3. The current state
4. The target state








**Example 1**: Rules-based Decision System

**Input Variables**:

In this case, these variables are observable only. This simplifies the problem for the example. Applied ot reinforcement learning, these elements may be conrollable. This is dependent on the problem being solved and is unique to every problem.

1. **Window Open**: (True/False) This is whether the window is open or closed.
2. **Outdoor Temperature**: (Float) The temperature outside (in degrees Fahrenheit).
3. **Indoor Temperature**: (Float) The current temperature inside the house (in degrees Fahrenheit).



**Output Variable**:

* For this example, the output is the thermostat temperature setting. However, this will also have constraints placed on it to define the system behavior.

* The target maximum indoor temperature is 70°F


**Constraints**:
* If the window is open, set the thermostat to match the indoor temperature. Opening the window might naturally regulate temperature without needing the thermostat to do much.
* If the window is closed:
  * If the indoor temperature is below 70°F (and the outdoor temperature is not excessively low, e.g., is above 30°F), set the thermostat to 70°F to heat the house.
  * If the indoor temperature is above 70°F and the outdoor temperature is also warm, set the thermostat to 70°F to cool down.
  * If the indoor temperature is above 70°F and the outdoor temperature is cool, set the thermostat to a lower temperature to save energy. (We'll use a set point of 65°F)



**General Solution**:

Given those inputs, the controllable output, and constraints, the rules might look something like:


**Condition**: Window is open

**Action**: Thermostat should match indoor temperature (don’t adjust).
    
<br>    
**Condition**: Window is closed and indoor temperature is below 70°F

**Action**: If outdoor temperature is not too low (e.g., above 30°F), set the thermostat to 70°F.

<br>

**Condition**: window is closed and indoor temperature is above 70°F

**Action 1**: If outdoor temperature is above 70°F, set the thermostat to 70°F.

**Action 2**: If outdoor temperature is below 70°F, set the thermostat to lower temperature (e.g., 65°F) to save energy.








In [1]:
def thermostat_decision(window_open, outdoor_temp, indoor_temp):
    # Target temperature
    target_temp = 70

    if window_open:
        # If window is open, do not change the thermostat, it should match indoor temperature
        return indoor_temp

    else:
        if indoor_temp < target_temp:
            # If indoor temperature is below the target, check outdoor temperature
            if outdoor_temp >= 30:
                # If outdoor temp is warm enough, set thermostat to target
                return target_temp
            else:
                # If outdoor temperature is too cold, set thermostat to warm enough temperature
                return target_temp

        elif indoor_temp > target_temp:
            # If indoor temperature is above the target, check outdoor temperature
            if outdoor_temp > target_temp:
                # If outdoor temperature is also warm, set thermostat to target
                return target_temp
            else:
                # If outdoor temperature is cool, set thermostat lower to save energy
                return 65

        # If indoor temperature is already at target, leave the thermostat as is
        return target_temp

# Example usage:

# Test Case 1: Window open, indoor temperature 72°F, outdoor temperature 50°F
print(thermostat_decision(True, 50, 72))  # Should return 72 (indoor temperature matches the thermostat)

# Test Case 2: Window closed, indoor temperature 68°F, outdoor temperature 75°F
print(thermostat_decision(False, 75, 68))  # Should return 70 (indoors below target, outdoor is warm)

# Test Case 3: Window closed, indoor temperature 72°F, outdoor temperature 65°F
print(thermostat_decision(False, 65, 72))  # Should return 65 (indoors too warm, outdoor is cooler)

# Test Case 4: Window closed, indoor temperature 60°F, outdoor temperature 50°F
print(thermostat_decision(False, 50, 60))  # Should return 70 (indoors below target, outdoor not too cold)


72
70
65
70


**Example 2**: Fuzzy Logic Decision System

Taking the same problem from example 1, this example reframes the 'rules' and variables in term of fuzzy sets and use fuzzy inference to determine the thermostat setting.


* **Fuzzy inputs** are the data or variables that feed into a fuzzy logic system. These inputs can be values that are numerical, or imprecise. In fuzzy logic, the inputs are often described in linguistic terms such as "cold," "hot," "moderate," or "warm." These terms are then mapped to fuzzy sets through membership functions.

* **Membership functions**  define how each input or output element is mapped to a fuzzy set. It assigns a degree of membership to a value based on how closely it matches a particular fuzzy set.

* **Fuzzy sets** are sets where each element has a degree of membership rather than just belonging to the set or not. In traditional (crisp) sets, an element either belongs to a set or does not. But in fuzzy sets, an element can belong to a set to a certain degree, expressed as a value between 0 and 1.

* **Fuzzy rules** are based on the desired behavior of the system. These are the logical statements that define how the fuzzy inputs interact with each other to produce a fuzzy output. These rules are generally structured as IF-THEN statements.

* **Fuzzy inference** combines the fuzzy inputs and applies the fuzzy rules to get the thermostat output.

Therefore, fuzzy logic allows systems to handle imprecision and partial truth, which is especially useful when dealing with human-like decision-making processes, where conditions are not always crisp (e.g., "hot" or "cold" might not have clear boundaries).

<br>

**Input Variables**:

In this case, these variables are observable only. This simplifies the problem for the example. Applied to reinforcement learning, these elements may be controllable. This is dependent on the problem being solved and is unique to every problem.

1. **Window Open**: (True/False) This is whether the window is open or closed.
2. **Outdoor Temperature**: (Float) The temperature outside (in degrees Fahrenheit).
3. **Indoor Temperature**: (Float) The current temperature inside the house (in degrees Fahrenheit).

**Output Variable**:

* For this example, the output is the thermostat temperature setting. However, this will also have constraints in the form of membership functions placed on it to control the system behavior.

* The target maximum indoor temperature is 70°F

**Fuzzy Sets**:
* The key variables are:
  * Window Open/Closed
  * Outdoor Temperature
  * Indoor Temperature
  * Thermostat Output (Temperature)

* Each of these variables needs to be mapped to fuzzy sets, with fuzzy membership functions that allow for degrees of belonging.

**Membership Functions**:

* Membership functions are defined by how categories overlap for variables.
* For temperature, these categories will be Low, Medium, High, and Very High
* For the window status, the fuzzy sets are Open or Closed.
* For the thermostat temperature output, the categories are Cold, Comfortable, and Hot.

<br>

Applied to the inputs, the fuzzy sets look like:
<br>

* Window Open/Closed:
   * Open: This means the window is fully open, but there could be degrees of "openness" (e.g., open halfway).
   * Closed: The window is completely closed, but there could be degrees of "closedness" (e.g., slightly open).

<br>

* Outdoor Temperature:
   * Cold: Below freezing, for example, 30°F or below.
   * Cool: Between 30°F and 50°F.
   * Moderate: Between 50°F and 70°F.
   * Warm: Between 70°F and 90°F.
   * Hot: Above 90°F.

<br>

* Indoor Temperature:
   * Cold: Below 60°F.
   * Comfortable: Between 60°F and 75°F.
   * Hot: Above 75°F.

<br>

* Thermostat Temperature (setpoint):
   * Cold: Lower temperature, e.g., 60°F.
   * Comfortable: The target setpoint, e.g., 70°F.
   * Hot: Higher temperature, e.g., 75°F or higher.


**Defining Fuzzy Rules**

Building off of the membership functions and fuzzy sets, the rules could look like:

* **Rule 1**: If the Window is Open, then the thermostat should be set to Comfortable (because the open window will adjust the temperature).
* **Rule 2**: If the Window is Closed and the Indoor Temperature is Cold, and the Outdoor Temperature is Cool or Moderate, then set the thermostat to Comfortable.
* **Rule 3**: If the Window is Closed and the Indoor Temperature is Hot, and the Outdoor Temperature is Warm or Hot, then set the thermostat to Comfortable.
* **Rule 4**: If the Window is Closed and the Indoor Temperature is Cold, and the Outdoor Temperature is Cold, then set the thermostat to Cold.
* **Rule 5**: If the Window is Closed and the Indoor Temperature is Hot, and the Outdoor Temperature is Cool or Cold, then set the thermostat to Cold.
* **Rule 6**: If the Window is Closed and the Indoor Temperature is Hot, and the Outdoor Temperature is Hot, then set the thermostat to Hot.


<br>
<br>

Due to how complex the implementation can be, this example uses the scikit-fuzzy library rather than sticking to built-in Python libraries or numpy as in previous tutorials.



In [3]:
!pip install scikit-fuzzy  #install the scikit-fuzzy library

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 920.8/920.8 kB 10.2 MB/s eta 0:00:00


In [4]:
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl

# 1. Define the fuzzy variables

# Input Variables
window_status = ctrl.Antecedent(np.arange(0, 11, 1), 'window_status')  # 0 = Closed, 10 = Open
outdoor_temp = ctrl.Antecedent(np.arange(0, 101, 1), 'outdoor_temp')  # Temperature in Fahrenheit
indoor_temp = ctrl.Antecedent(np.arange(0, 101, 1), 'indoor_temp')

# Output Variable
thermostat_temp = ctrl.Consequent(np.arange(60, 81, 1), 'thermostat_temp')  # Thermostat setting in Fahrenheit

# 2. Define the fuzzy sets (membership functions)
# window
window_status['closed'] = fuzz.trimf(window_status.universe, [0, 0, 5])
window_status['open'] = fuzz.trimf(window_status.universe, [5, 10, 10])
# outoor temperature
outdoor_temp['cold'] = fuzz.trimf(outdoor_temp.universe, [0, 30, 50])
outdoor_temp['moderate'] = fuzz.trimf(outdoor_temp.universe, [50, 70, 90])
outdoor_temp['hot'] = fuzz.trimf(outdoor_temp.universe, [70, 90, 100])
#indoor temperature
indoor_temp['cold'] = fuzz.trimf(indoor_temp.universe, [0, 60, 70])
indoor_temp['comfortable'] = fuzz.trimf(indoor_temp.universe, [60, 70, 75])
indoor_temp['hot'] = fuzz.trimf(indoor_temp.universe, [70, 80, 100])
# thermostat output temperature.
# these values attempt to stay between the minimum value of 65*F in the previous example,
# and a maximum value of 80 in order to not make the 'hot' set too narrow.
# however, these should be expanded to include a broader ranger of temperatures
# in more realistic examples
thermostat_temp['cold'] = fuzz.trimf(thermostat_temp.universe, [60, 65, 70])
thermostat_temp['comfortable'] = fuzz.trimf(thermostat_temp.universe, [65, 70, 75])
thermostat_temp['hot'] = fuzz.trimf(thermostat_temp.universe, [70, 75, 80])

# 3. Define fuzzy rules
rule1 = ctrl.Rule(window_status['open'], thermostat_temp['comfortable'])
rule2 = ctrl.Rule(window_status['closed'] & (indoor_temp['cold'] & outdoor_temp['moderate']), thermostat_temp['comfortable'])
rule3 = ctrl.Rule(window_status['closed'] & (indoor_temp['hot'] & outdoor_temp['moderate']), thermostat_temp['comfortable'])
rule4 = ctrl.Rule(window_status['closed'] & (indoor_temp['cold'] & outdoor_temp['cold']), thermostat_temp['cold'])
rule5 = ctrl.Rule(window_status['closed'] & (indoor_temp['hot'] & outdoor_temp['cold']), thermostat_temp['cold'])
rule6 = ctrl.Rule(window_status['closed'] & (indoor_temp['hot'] & outdoor_temp['hot']), thermostat_temp['hot'])

# 4. Create the control system and simulation
thermostat_ctrl = ctrl.ControlSystem([rule1, rule2, rule3, rule4, rule5, rule6])
thermostat = ctrl.ControlSystemSimulation(thermostat_ctrl)

# 5. Examples!
# Example 1: Window Open, Indoor Temperature = 72°F, Outdoor Temperature = 50°F
thermostat.input['window_status'] = 10  # Window fully open
thermostat.input['outdoor_temp'] = 50
thermostat.input['indoor_temp'] = 72
thermostat.compute()
print(f'Thermostat setting: {thermostat.output["thermostat_temp"]:.2f}°F')

# Example 2: Window Closed, Indoor Temperature = 60°F, Outdoor Temperature = 75°F
thermostat.input['window_status'] = 0  # Window closed
thermostat.input['outdoor_temp'] = 75
thermostat.input['indoor_temp'] = 60
thermostat.compute()
print(f'Thermostat setting: {thermostat.output["thermostat_temp"]:.2f}°F')


Thermostat setting: 70.00°F
Thermostat setting: 70.00°F
